# Data Processing

In [1]:
%%time
!pip3 freeze | grep -E 'boto3|s3fs|nltk|distributed|dask==|dask-m|black==|jupyter-server|pandas'
!conda list -n spark | grep -E 'ipykernel|openjdk|pyspark'

black==22.6.0
boto3==1.24.56
dask==2022.8.0
dask-ml==2022.5.27
distributed==2022.8.0
nb-black==1.0.7
nltk==3.7
pandas==1.4.3
s3fs==0.4.2
ipykernel                 6.15.1             pyh210e3f2_0    conda-forge
openjdk                   17.0.3               h1e1ecb3_1    conda-forge
pyspark                   3.3.0              pyhd8ed1ab_0    conda-forge
CPU times: user 34.3 ms, sys: 24.5 ms, total: 58.8 ms
Wall time: 2.51 s


In [2]:
%load_ext lab_black

In [3]:
import os
import shutil
from datetime import datetime
from functools import reduce
from glob import glob
import zipfile

import boto3
import dask.dataframe as dd
import pandas as pd
from pyspark import SparkConf, SparkFiles
import pyspark.pandas as ps
from pyspark.sql import Column, SparkSession, functions as F, types as T
from pyspark.sql.dataframe import DataFrame as pdf
from pyspark.sql.window import Window

## About

The streamed tweet data and metadata will  now be loaded with PySpark and processed for use in sentiment classification with a big-data ML framework
- load all the hourly `.parquet` data files saved to S3 objects in the `/processed` prefix, into a single PySpark DataFrame
- perform the following data processing on the `text` of the tweet using PySpark
  - download `.parquet` file from S3
  - extract all contained `.parquet` files (without subfolders)
  - read all `.parquet` files into single PySpark DataFrame
  - process data using PySpark
    - drop leading and trailing whitespaces
    - replace occurrences of multiple consecutive whitespaces by a single whitespace
    - drop tweets with missing (`NaN`s) or blank (`''`) `text`
    - change to lowercase
    - remove special characters
    - remove numbers
    - remove punctuation
  - (optional) filter the data to remove short tweets (number of words below a user-defined threshold)

  and save the processed tweet `text` in a separate column (named `text_processed`) from the original `text`
- save the data after processing and filtering to a `.parquet` file on S3 in the `/processed` prefix

### Pre-Requisites
1. This notebook has been run from inside a `conda` environment on a system Python version of 3.10, with the following Python libraries installed
   ```bash
   - pip==22.2.2
   - pip:
     - nb_black==1.0.7
     - boto3
     - s3fs
     - nltk==3.7
     - distributed==2022.8.0
     - dask[dataframe]==2022.8.0
     - dask[distributed]==2022.8.0
     - dask-ml==2022.5.27
   - conda==4.3.16
   - conda:
     - ipykernel==6.15.1
     - openjdk==17.0.3
     - pyspark==3.3.0
    ```
2. If this notebook is being run on a Sagemaker notebook instance, then only `nltk` should need to be installed manually using
   ```bash
   !pip3 install nltk==3.7
   ```

   in the first code cell of this notebook.

## User Inputs

In [4]:
path_to_folder = "/datasets/twitter/kinesis-demo/"

# data with no unwanted terms in tweets
processed_data_dir = "data/processed"
proc_zip_fname = "processed_data.zip"
# cols_to_load = [
#     "id",
#     "contributors",
#     "created_at" "source",
#     "in_reply_to_screen_name",
#     "source_text",
#     "created_at",
#     "place_id",
#     "place_url",
#     "place_place_type",
#     "place_country_code",
#     "place_country",
#     "user_name",
#     "user_screen_name",
#     "user_followers",
#     "user_friends",
#     "user_listed",
#     "user_favourites",
#     "user_statuses",
#     "user_protected" "user_verified",
#     "user_joined",
#     "user_location",
#     "retweeted_tweet",
#     "is_quote_status",
#     "quote_count",
#     "reply_count",
#     "favorite_count",
#     "favorited",
#     "text",
# ]

# processed data
processed_file_name = "processed_text"
min_num_words_tweet = 10

upload_to_s3 = True
cleanup_local_files = True

In [5]:
s3_bucket_name = os.getenv("AWS_S3_BUCKET_NAME", "sagemakertestwillz3s")
session = boto3.Session(profile_name="default")
s3_client = session.client("s3")

dtypes_dict = {
    "id": pd.StringDtype(),
    "geo": pd.StringDtype(),
    "coordinates": pd.StringDtype(),
    "place": pd.StringDtype(),
    "contributors": pd.StringDtype(),  # pd.BooleanDtype(),
    "is_quote_status": pd.StringDtype(),  # pd.BooleanDtype(),
    "quote_count": pd.Int32Dtype(),
    "reply_count": pd.Int32Dtype(),
    "retweet_count": pd.Int32Dtype(),
    "favorite_count": pd.Int32Dtype(),
    "favorited": pd.StringDtype(),  # pd.BooleanDtype(),
    "retweeted": pd.StringDtype(),  # pd.BooleanDtype(),
    "source": pd.StringDtype(),
    "in_reply_to_user_id": pd.StringDtype(),
    "in_reply_to_screen_name": pd.StringDtype(),
    "source_text": pd.StringDtype(),
    "place_id": pd.StringDtype(),
    "place_url": pd.StringDtype(),
    "place_place_type": pd.StringDtype(),
    "place_name": pd.StringDtype(),
    "place_full_name": pd.StringDtype(),
    "place_country_code": pd.StringDtype(),
    "place_country": pd.StringDtype(),
    "place_bounding_box_type": pd.StringDtype(),
    "place_bounding_box_coordinates": pd.StringDtype(),
    "place_attributes": pd.StringDtype(),
    "coords_type": pd.StringDtype(),
    "coords_lon": pd.StringDtype(),
    "coords_lat": pd.StringDtype(),
    "geo_type": pd.StringDtype(),
    "geo_lon": pd.StringDtype(),
    "geo_lat": pd.StringDtype(),
    "user_name": pd.StringDtype(),
    "user_screen_name": pd.StringDtype(),
    "user_followers": pd.Int32Dtype(),
    "user_friends": pd.Int32Dtype(),
    "user_listed": pd.Int32Dtype(),
    "user_favourites": pd.Int32Dtype(),
    "user_statuses": pd.Int32Dtype(),
    "user_protected": pd.StringDtype(),  # pd.BooleanDtype(),
    "user_verified": pd.StringDtype(),  # pd.BooleanDtype(),
    "user_contributors_enabled": pd.StringDtype(),
    "user_location": pd.StringDtype(),
    "retweeted_tweet": pd.StringDtype(),
    "tweet_text_urls": pd.StringDtype(),
    "tweet_text_hashtags": pd.StringDtype(),
    "tweet_text_usernames": pd.StringDtype(),
    "num_urls_in_tweet_text": pd.Int32Dtype(),
    "num_users_in_tweet_text": pd.Int32Dtype(),
    "num_hashtags_in_tweet_text": pd.Int32Dtype(),
    "text": pd.StringDtype(),
    "contains_wanted_text": pd.BooleanDtype(),
    "contains_wanted_text_case_sensitive": pd.BooleanDtype(),
    "contains_multi_word_wanted_text": pd.BooleanDtype(),
    "contains_crypto_terms": pd.BooleanDtype(),
    "contains_religious_terms": pd.BooleanDtype(),
    "contains_inappropriate_terms": pd.BooleanDtype(),
    "contains_video_games_terms": pd.BooleanDtype(),
    "contains_misc_unwanted_terms": pd.BooleanDtype(),
    "contains_non_english_terms": pd.BooleanDtype(),
}

proc_text_zip_fname = f"{processed_file_name}.zip"

In [6]:
def show_pyspark_df(df: pdf, nrows: int = 5) -> pd.DataFrame:
    """Display the first n rows of a PySpark DataFrame as a Pandas DataFrame."""
    return df.limit(nrows).toPandas()


def create_zip_file_from_folder(
    processed_data_dir: str, zip_fname: str, filepath: str
) -> None:
    """Create zipped file from a folder."""
    # create a ZipFile object
    with zipfile.ZipFile(os.path.join(processed_data_dir, zip_fname), "w") as zipObj:
        # Iterate over all the files in directory
        for folderName, subfolders, filenames in os.walk(filepath):
            for filename in filenames:
                # print(folderName, filename)
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # # Add file to zip
                zipObj.write(filePath, os.path.basename(filePath))


def upload_file_to_s3(
    aws_region: str,
    processed_data_dir: str,
    fname: str,
    s3_bucket_name: str,
    s3_key: str,
) -> None:
    """Upload file to key in S3 bucket."""
    s3_resource = boto3.resource("s3", region_name=aws_region)
    s3_resource.meta.client.upload_file(
        f"{processed_data_dir}/{fname}",
        s3_bucket_name,
        s3_key,
    )


def download_file_from_s3(
    s3_bucket_name: str,
    path_to_folder: str,
    data_dir: str,
    fname: str,
    aws_region: str,
    prefix: str,
) -> None:
    """Download file from ."""
    dest_filepath = os.path.join(data_dir, fname)
    s3_filepath_key = s3_client.list_objects_v2(
        Bucket=s3_bucket_name,
        Delimiter="/",
        Prefix=prefix,
    )["Contents"][0]["Key"]
    start = datetime.now()
    print(
        f"Started downloading processed data zip file from {s3_filepath_key} to "
        f"{dest_filepath} at {start.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]}..."
    )
    s3 = boto3.resource("s3", region_name=aws_region)
    s3.meta.client.download_file(
        s3_bucket_name,
        s3_filepath_key,
        dest_filepath,
    )
    duration = (datetime.now() - start).total_seconds()
    print(f"Done downloading in {duration:.3f} seconds.")


def extract_zip_file(dest_filepath: str, data_dir: str) -> None:
    """."""
    start = datetime.now()
    print(
        "Started extracting filtered data parquet files from "
        f"processed data zip file to {data_dir} at "
        f"{start.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]}..."
    )
    zip_ref = zipfile.ZipFile(dest_filepath)
    zip_ref.extractall(data_dir)
    zip_ref.close()
    duration = (datetime.now() - start).total_seconds()
    print(f"Done extracting in {duration:.3f} seconds.")

## PySpark Setup

In [7]:
%%time
conf = (SparkConf())

CPU times: user 14 µs, sys: 0 ns, total: 14 µs
Wall time: 17.9 µs


In [8]:
# .set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars()))

Start a Spark session

In [9]:
%%time
spark = (
    SparkSession
    .builder
    .config(conf=conf)
    .appName("schema_test")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/26 15:20:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
CPU times: user 25.5 ms, sys: 8.35 ms, total: 33.9 ms
Wall time: 3.89 s


## Get Data

We will start by downloaded the processed `.zip` file from S3 and extracting all the contained `.parquet` files into the same directory as the `.zip` file

In [10]:
%%time
if not os.path.exists(os.path.join(processed_data_dir, proc_zip_fname)):
    download_file_from_s3(
        s3_bucket_name,
        path_to_folder,
        processed_data_dir,
        proc_zip_fname,
        session.region_name,
        f"{path_to_folder[1:]}processed/{proc_zip_fname}",
    )
    extract_zip_file(os.path.join(processed_data_dir, proc_zip_fname), processed_data_dir)
proc_files = glob(f"{processed_data_dir}/*.parquet.gzip")

Started downloading processed data zip file from datasets/twitter/kinesis-demo/processed/processed_data.zip to data/processed/processed_data.zip at 2022-08-26 15:20:51.312...
Done downloading in 0.322 seconds.
Started extracting filtered data parquet files from processed data zip file to data/processed at 2022-08-26 15:20:51.636...
Done extracting in 0.405 seconds.
CPU times: user 424 ms, sys: 146 ms, total: 569 ms
Wall time: 773 ms


We will now use the `pandas` API on Spark to load all the `.parquet` files into a single PySpark DataFrame. This could equivalently be done using native PySpark capabilities with
```python
def Zconcat(dfs):
    """Concatenate two PySpark DataFrames."""
    return reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)


def union_all(dfs):
    """Concatenate multiple PySpark DataFrames."""
    columns = reduce(lambda x, y: set(x).union(set(y)), [i.columns for i in dfs])

    for i in range(len(dfs)):
        d = dfs[i]
        for c in columns:
            if c not in d.columns:
                d = d.withColumn(c, lit(None))
        dfs[i] = d

    return Zconcat(dfs)


df = union_all([spark.read.parquet(f) for f in proc_files])
with pd.option_context("display.max_columns", None):
    display(show_pyspark_df(df))
```

but we will instead perform this using the `pandas` API on Spark, as shown below

In [11]:
%%time
df = (
    ps.read_parquet(proc_files, index_col=['id'])
    .reset_index()
    .astype(dtypes_dict)
)
with pd.option_context("display.max_columns", None):
    display(df.head())

22/08/26 15:21:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,id,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,favorited,retweeted,created_at,source,in_reply_to_user_id,in_reply_to_screen_name,source_text,place_id,place_url,place_place_type,place_name,place_full_name,place_country_code,place_country,place_bounding_box_type,place_bounding_box_coordinates,place_attributes,coords_type,coords_lon,coords_lat,geo_type,geo_lon,geo_lat,user_name,user_screen_name,user_followers,user_friends,user_listed,user_favourites,user_statuses,user_protected,user_verified,user_contributors_enabled,user_joined,user_location,retweeted_tweet,tweet_text_urls,tweet_text_hashtags,tweet_text_usernames,num_urls_in_tweet_text,num_users_in_tweet_text,num_hashtags_in_tweet_text,text,contains_wanted_text,contains_wanted_text_case_sensitive,contains_multi_word_wanted_text,contains_crypto_terms,contains_religious_terms,contains_inappropriate_terms,contains_video_games_terms,contains_misc_unwanted_terms,contains_non_english_terms
0,1479875656062222347,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:25,"<a href=""http://twitter.com/download/iphone"" r...",None,None,Twitter for iPhone,,,,,,,,,[[]],{},,,,,,,Scherri Evans 🪐,ScherriEvans,180,209,3,21706,14009,False,False,False,2015-06-09 18:12:49,Mars,no,https://t.co/fvYJuLrkKE|https://t.co/4ovbJKdU1F,,NASAWebb,2,1,0,Today the first mirror wing on swung into pla...,False,False,True,False,False,False,False,False,False
1,1479875662878019585,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:27,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,Twitter Web App,,,,,,,,,[[]],{},,,,,,,MURTWITNESSONE,MURTWITNESSONE,3506,5002,69,521,350862,False,False,False,2009-03-19 22:25:26,"Orlando, FL",no,,,,0,0,0,Elon Musk gains 117k Twitter followers per day...,False,True,False,False,False,False,False,False,False
2,1479875694649819141,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:35,"<a href=""http://twitter.com/download/iphone"" r...",None,None,Twitter for iPhone,,,,,,,,,[[]],{},,,,,,,JP iV,jP_iV4,41,222,0,909,473,False,False,False,2015-09-23 21:02:11,"Pittsburgh, PA",no,https://t.co/Kv57EeVruf,,,1,0,0,"“What an amazing milestone."" NASA's new space ...",False,True,True,False,False,False,False,False,False
3,1479875700530290692,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:36,"<a href=""http://twitter.com/download/android"" ...",None,None,Twitter for Android,,,,,,,,,[[]],{},,,,,,,Cynthia Simmons,cyntsimmons,209,265,0,12120,4119,False,False,False,2013-04-11 01:00:46,None,no,https://t.co/WWhOOqSn0w,,,1,0,0,"""The observatory launched April 24, 1990, Hubb...",True,False,False,False,False,False,False,False,False
4,1479875703411728391,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:37,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,Twitter Web App,,,,,,,,,[[]],{},,,,,,,ari4,a4floridacajun,2193,3683,6,69635,72063,False,False,False,2019-01-24 00:38:11,"Niceville, FL",no,https://t.co/Kv57EeVruf,,,1,0,0,"“What an amazing milestone."" NASA's new space ...",False,True,True,False,False,False,False,False,False


CPU times: user 448 ms, sys: 106 ms, total: 554 ms
Wall time: 12.9 s


The column datatypes are shown below

In [12]:
%%time
with pd.option_context("display.max_rows", None):
    display(df.dtypes.rename("dtype").to_frame())

,dtype
id,string
geo,string
coordinates,string
place,string
contributors,string
is_quote_status,string
quote_count,Int32
reply_count,Int32
retweet_count,Int32
favorite_count,Int32


CPU times: user 20.6 ms, sys: 4.74 ms, total: 25.4 ms
Wall time: 25.2 ms


Get the number of rows (retrieved tweets) in the data, number of pyspark `DataFrame` partitions and the number of workers on the host (single-node) cluster

In [13]:
%%time
print(
    f"Raw data contains {len(df):,} rows and {len(df.columns):,} columns "
    f"on a host with {len(os.sched_getaffinity(0))} CPUs"
)

Raw data contains 171,506 rows and 62 columns on a host with 4 CPUs
CPU times: user 2.66 ms, sys: 448 µs, total: 3.11 ms
Wall time: 1.19 s


## Process Data

Process the tweet text using the `pandas` API on `PySpark`

In [14]:
%%time
df_processed = (
    df
    # drop tweets with NaN in the text column
    .dropna(subset=['text'])
    # remove leading, trailing spaces and multiple spaces with single space
    .assign(text_stripped=lambda df: df['text'].str.lstrip().str.rstrip().str.replace(r"\s+", " "))
    # remove blank tweets
    .query("text_stripped != ''")
    .assign(
        text_processed=lambda df: df['text_stripped']
        # lowercase
        .str.lower()
        # remove special characters
        .str.replace(r"[^a-zA-z]", " ")
        # remove numbers
        .str.replace(r"\d+", " ")
        # remove punctuation
        .str.replace('[^\w\s]', '')
    )
)
print(
    f"Processed data contains {len(df_processed):,} rows and "
    f"{len(df_processed.columns):,} columns."
)
with pd.option_context("display.max_columns", None, "display.max_colwidth", None):
    display(df_processed[['id', 'source_text', 'text', 'text_stripped', 'text_processed']].head(2))

Processed data contains 171,506 rows and 64 columns.


,id,source_text,text,text_stripped,text_processed
0,1479875656062222347,Twitter for iPhone,"Today the first mirror wing on swung into place, but did you know that work ensuring the mirrors were perfectly polished led to tools that help diagnose eye conditions and improve Lasik surgery? Read about this and more space telescope spinoffs:","Today the first mirror wing on swung into place, but did you know that work ensuring the mirrors were perfectly polished led to tools that help diagnose eye conditions and improve Lasik surgery? Read about this and more space telescope spinoffs:",today the first mirror wing on swung into place but did you know that work ensuring the mirrors were perfectly polished led to tools that help diagnose eye conditions and improve lasik surgery read about this and more space telescope spinoffs
1,1479875662878019585,Twitter Web App,Elon Musk gains 117k Twitter followers per day on average last 30 days. NASA gains 26k Twitter followers per day on average last 30 days. Bill Gates gains 10k Twitter followers per day on average last 30 days.,Elon Musk gains 117k Twitter followers per day on average last 30 days. NASA gains 26k Twitter followers per day on average last 30 days. Bill Gates gains 10k Twitter followers per day on average last 30 days.,elon musk gains k twitter followers per day on average last days nasa gains k twitter followers per day on average last days bill gates gains k twitter followers per day on average last days


CPU times: user 173 ms, sys: 45.8 ms, total: 219 ms
Wall time: 17.2 s


The distribution of original tweets and re-tweets is shown below

In [15]:
%%time
display(
    df_processed['retweeted_tweet']
    .value_counts()
    .rename("num_tweets")
    .reset_index()
    .rename(columns={"index": "retweeted_tweet"}).merge(
        df_processed['retweeted_tweet']
        .value_counts(normalize=True)
        .rename("frac_tweets")
        .reset_index()
        .rename(columns={"index": "retweeted_tweet"}),
        on="retweeted_tweet",
        how="left",
    )
)

,retweeted_tweet,num_tweets,frac_tweets
0,no,171295,0.99877
1,yes,211,0.00123


CPU times: user 98 ms, sys: 24.2 ms, total: 122 ms
Wall time: 14.3 s


**Notes**
1. More than 99% of valid tweets for this use-case are original tweets. For classifying sentiment, we can keep the retweets, but retweets are not needed when extracting the sentiment (in the `7_*.ipynb` notebook), so we'll need to drop them during sentiment extraction. For now, we will leave the retweets in the processed data.

We'll now extract the approximate number of words in each *original* tweet (not the processed tweet). We will use this to optionally filter the dataset to remove short tweets (whose length is below some threshold we specify). To get the words, we'll split the text on whitespace. Since the `pandas` API on PySpark does not support splitting a string based on multiple occurrences of a separator, we'll convert this `DataFrame` to PySpark and use PySpark-native methods to perfrom this split.

Below, we convert the `pandas` on PySpark DataFrame to a PySpark DataFrame

In [16]:
%%time
dfpy = df_processed.set_index('id').to_spark(index_col='id')
with pd.option_context("display.max_columns", None):
    display(show_pyspark_df(dfpy, 2))

,id,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,favorited,retweeted,created_at,source,in_reply_to_user_id,in_reply_to_screen_name,source_text,place_id,place_url,place_place_type,place_name,place_full_name,place_country_code,place_country,place_bounding_box_type,place_bounding_box_coordinates,place_attributes,coords_type,coords_lon,coords_lat,geo_type,geo_lon,geo_lat,user_name,user_screen_name,user_followers,user_friends,user_listed,user_favourites,user_statuses,user_protected,user_verified,user_contributors_enabled,user_joined,user_location,retweeted_tweet,tweet_text_urls,tweet_text_hashtags,tweet_text_usernames,num_urls_in_tweet_text,num_users_in_tweet_text,num_hashtags_in_tweet_text,text,contains_wanted_text,contains_wanted_text_case_sensitive,contains_multi_word_wanted_text,contains_crypto_terms,contains_religious_terms,contains_inappropriate_terms,contains_video_games_terms,contains_misc_unwanted_terms,contains_non_english_terms,text_stripped,text_processed
0,1479875656062222347,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:25,"<a href=""http://twitter.com/download/iphone"" r...",None,None,Twitter for iPhone,,,,,,,,,[[]],{},,,,,,,Scherri Evans 🪐,ScherriEvans,180,209,3,21706,14009,False,False,False,2015-06-09 18:12:49,Mars,no,https://t.co/fvYJuLrkKE|https://t.co/4ovbJKdU1F,,NASAWebb,2,1,0,Today the first mirror wing on swung into pla...,False,False,True,False,False,False,False,False,False,Today the first mirror wing on swung into plac...,today the first mirror wing on swung into plac...
1,1479875662878019585,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:27,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,Twitter Web App,,,,,,,,,[[]],{},,,,,,,MURTWITNESSONE,MURTWITNESSONE,3506,5002,69,521,350862,False,False,False,2009-03-19 22:25:26,"Orlando, FL",no,,,,0,0,0,Elon Musk gains 117k Twitter followers per day...,False,True,False,False,False,False,False,False,False,Elon Musk gains 117k Twitter followers per day...,elon musk gains k twitter followers per day...


CPU times: user 77.9 ms, sys: 785 µs, total: 78.7 ms
Wall time: 9.12 s


We now use a PySpark native function `.split()` to extract the words from the `text` of the tweet

In [17]:
%%time
dfpy = dfpy.withColumn(
    "text_trimmed", F.trim(F.col("text"))
).withColumn("words", F.split("text_trimmed", "\s+"))
print(f"Number of rows in processed data = {dfpy.count():,}")

Number of rows in processed data = 171,506
CPU times: user 14.4 ms, sys: 3.3 ms, total: 17.7 ms
Wall time: 6.41 s


In [18]:
%%time
print(
    f"Processed data contains {dfpy.count():,} rows and {len(dfpy.columns):,} columns "
    f"in {dfpy.rdd.getNumPartitions()} partitions, on a host with "
    f"{len(os.sched_getaffinity(0))} CPUs"
)
with pd.option_context("display.max_columns", None):
    display(show_pyspark_df(dfpy, 2))

Processed data contains 171,506 rows and 66 columns in 8 partitions, on a host with 4 CPUs


,id,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,favorited,retweeted,created_at,source,in_reply_to_user_id,in_reply_to_screen_name,source_text,place_id,place_url,place_place_type,place_name,place_full_name,place_country_code,place_country,place_bounding_box_type,place_bounding_box_coordinates,place_attributes,coords_type,coords_lon,coords_lat,geo_type,geo_lon,geo_lat,user_name,user_screen_name,user_followers,user_friends,user_listed,user_favourites,user_statuses,user_protected,user_verified,user_contributors_enabled,user_joined,user_location,retweeted_tweet,tweet_text_urls,tweet_text_hashtags,tweet_text_usernames,num_urls_in_tweet_text,num_users_in_tweet_text,num_hashtags_in_tweet_text,text,contains_wanted_text,contains_wanted_text_case_sensitive,contains_multi_word_wanted_text,contains_crypto_terms,contains_religious_terms,contains_inappropriate_terms,contains_video_games_terms,contains_misc_unwanted_terms,contains_non_english_terms,text_stripped,text_processed,text_trimmed,words
0,1479875656062222347,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:25,"<a href=""http://twitter.com/download/iphone"" r...",None,None,Twitter for iPhone,,,,,,,,,[[]],{},,,,,,,Scherri Evans 🪐,ScherriEvans,180,209,3,21706,14009,False,False,False,2015-06-09 18:12:49,Mars,no,https://t.co/fvYJuLrkKE|https://t.co/4ovbJKdU1F,,NASAWebb,2,1,0,Today the first mirror wing on swung into pla...,False,False,True,False,False,False,False,False,False,Today the first mirror wing on swung into plac...,today the first mirror wing on swung into plac...,Today the first mirror wing on swung into pla...,"[Today, the, first, mirror, wing, on, swung, i..."
1,1479875662878019585,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:27,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,Twitter Web App,,,,,,,,,[[]],{},,,,,,,MURTWITNESSONE,MURTWITNESSONE,3506,5002,69,521,350862,False,False,False,2009-03-19 22:25:26,"Orlando, FL",no,,,,0,0,0,Elon Musk gains 117k Twitter followers per day...,False,True,False,False,False,False,False,False,False,Elon Musk gains 117k Twitter followers per day...,elon musk gains k twitter followers per day...,Elon Musk gains 117k Twitter followers per day...,"[Elon, Musk, gains, 117k, Twitter, followers, ..."


CPU times: user 114 ms, sys: 8.54 ms, total: 122 ms
Wall time: 15.1 s


We'll now convert the DataFrame back to the `pandas` API on PySpark and continue with data processing where we will count the number of words in the `words` column

In [19]:
%%time
df_words = dfpy.pandas_api().assign(num_words=lambda df: df["words"].str.len())
display(df_words.head(4))

,id,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,favorited,retweeted,created_at,source,in_reply_to_user_id,in_reply_to_screen_name,source_text,place_id,place_url,place_place_type,place_name,place_full_name,place_country_code,place_country,place_bounding_box_type,place_bounding_box_coordinates,place_attributes,coords_type,coords_lon,coords_lat,geo_type,geo_lon,geo_lat,user_name,user_screen_name,user_followers,user_friends,user_listed,user_favourites,user_statuses,user_protected,user_verified,user_contributors_enabled,user_joined,user_location,retweeted_tweet,tweet_text_urls,tweet_text_hashtags,tweet_text_usernames,num_urls_in_tweet_text,num_users_in_tweet_text,num_hashtags_in_tweet_text,text,contains_wanted_text,contains_wanted_text_case_sensitive,contains_multi_word_wanted_text,contains_crypto_terms,contains_religious_terms,contains_inappropriate_terms,contains_video_games_terms,contains_misc_unwanted_terms,contains_non_english_terms,text_stripped,text_processed,text_trimmed,words,num_words
0,1479875656062222347,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:25,"<a href=""http://twitter.com/download/iphone"" r...",None,None,Twitter for iPhone,,,,,,,,,[[]],{},,,,,,,Scherri Evans 🪐,ScherriEvans,180,209,3,21706,14009,False,False,False,2015-06-09 18:12:49,Mars,no,https://t.co/fvYJuLrkKE|https://t.co/4ovbJKdU1F,,NASAWebb,2,1,0,Today the first mirror wing on swung into pla...,False,False,True,False,False,False,False,False,False,Today the first mirror wing on swung into plac...,today the first mirror wing on swung into plac...,Today the first mirror wing on swung into pla...,"[Today, the, first, mirror, wing, on, swung, i...",41
1,1479875662878019585,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:27,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,Twitter Web App,,,,,,,,,[[]],{},,,,,,,MURTWITNESSONE,MURTWITNESSONE,3506,5002,69,521,350862,False,False,False,2009-03-19 22:25:26,"Orlando, FL",no,,,,0,0,0,Elon Musk gains 117k Twitter followers per day...,False,True,False,False,False,False,False,False,False,Elon Musk gains 117k Twitter followers per day...,elon musk gains k twitter followers per day...,Elon Musk gains 117k Twitter followers per day...,"[Elon, Musk, gains, 117k, Twitter, followers, ...",38
2,1479875694649819141,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:35,"<a href=""http://twitter.com/download/iphone"" r...",None,None,Twitter for iPhone,,,,,,,,,[[]],{},,,,,,,JP iV,jP_iV4,41,222,0,909,473,False,False,False,2015-09-23 21:02:11,"Pittsburgh, PA",no,https://t.co/Kv57EeVruf,,,1,0,0,"“What an amazing milestone."" NASA's new space ...",False,True,True,False,False,False,False,False,False,"“What an amazing milestone."" NASA's new space ...",what an amazing milestone nasa s new space ...,"“What an amazing milestone."" NASA's new space ...","[“What, an, amazing, milestone."", NASA's, new,...",35
3,1479875700530290692,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:36,"<a href=""http://twitter.com/download/android"" ...",None,None,Twitter for Android,,,,,,,,,[[]],{},,,,,,,Cynthia Simmons,cyntsimmons,209,265,0,12120,4119,False,False,False,2013-04-11 01:00:46,None,no,https://t.co/WWhOOqSn0w,,,1,0,0,"""The observatory launched April 24, 1990, Hubb...",True,False,False,False,False,False,False,False,False,"""The observatory launched April 24, 1990, Hubb...",the observatory launched april hubb...,"""The observatory launched April 24, 1990, Hubb...","[""The, observatory, launched, April, 24,, 1990...",37


CPU times: user 376 ms, sys: 53.9 ms, total: 430 ms
Wall time: 36.3 s


We can now optionally filter the data based on the number of words. We will choose to keep tweets that are of a minimum length of 10 words (controlled by the `min_num_words` variable in the **User Inputs** section) and drop shorter ones

In [20]:
%%time
df_words_long = df_words.query(f"num_words >= {min_num_words_tweet}")
print(
    "Number of rows in processed data, after filtering out tweets based on "
    f"length of text = {len(df_words_long):,}"
)
display(df_words_long.head(6))

Number of rows in processed data, after filtering out tweets based on length of text = 169,872


22/08/26 15:23:13 WARN MemoryStore: Not enough space to cache rdd_368_4 in memory! (computed 31.7 MiB so far)
22/08/26 15:23:13 WARN BlockManager: Persisting block rdd_368_4 to disk instead.


,id,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,favorited,retweeted,created_at,source,in_reply_to_user_id,in_reply_to_screen_name,source_text,place_id,place_url,place_place_type,place_name,place_full_name,place_country_code,place_country,place_bounding_box_type,place_bounding_box_coordinates,place_attributes,coords_type,coords_lon,coords_lat,geo_type,geo_lon,geo_lat,user_name,user_screen_name,user_followers,user_friends,user_listed,user_favourites,user_statuses,user_protected,user_verified,user_contributors_enabled,user_joined,user_location,retweeted_tweet,tweet_text_urls,tweet_text_hashtags,tweet_text_usernames,num_urls_in_tweet_text,num_users_in_tweet_text,num_hashtags_in_tweet_text,text,contains_wanted_text,contains_wanted_text_case_sensitive,contains_multi_word_wanted_text,contains_crypto_terms,contains_religious_terms,contains_inappropriate_terms,contains_video_games_terms,contains_misc_unwanted_terms,contains_non_english_terms,text_stripped,text_processed,text_trimmed,words,num_words
0,1479875656062222347,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:25,"<a href=""http://twitter.com/download/iphone"" r...",None,None,Twitter for iPhone,,,,,,,,,[[]],{},,,,,,,Scherri Evans 🪐,ScherriEvans,180,209,3,21706,14009,False,False,False,2015-06-09 18:12:49,Mars,no,https://t.co/fvYJuLrkKE|https://t.co/4ovbJKdU1F,,NASAWebb,2,1,0,Today the first mirror wing on swung into pla...,False,False,True,False,False,False,False,False,False,Today the first mirror wing on swung into plac...,today the first mirror wing on swung into plac...,Today the first mirror wing on swung into pla...,"[Today, the, first, mirror, wing, on, swung, i...",41
1,1479875662878019585,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:27,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,Twitter Web App,,,,,,,,,[[]],{},,,,,,,MURTWITNESSONE,MURTWITNESSONE,3506,5002,69,521,350862,False,False,False,2009-03-19 22:25:26,"Orlando, FL",no,,,,0,0,0,Elon Musk gains 117k Twitter followers per day...,False,True,False,False,False,False,False,False,False,Elon Musk gains 117k Twitter followers per day...,elon musk gains k twitter followers per day...,Elon Musk gains 117k Twitter followers per day...,"[Elon, Musk, gains, 117k, Twitter, followers, ...",38
2,1479875694649819141,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:35,"<a href=""http://twitter.com/download/iphone"" r...",None,None,Twitter for iPhone,,,,,,,,,[[]],{},,,,,,,JP iV,jP_iV4,41,222,0,909,473,False,False,False,2015-09-23 21:02:11,"Pittsburgh, PA",no,https://t.co/Kv57EeVruf,,,1,0,0,"“What an amazing milestone."" NASA's new space ...",False,True,True,False,False,False,False,False,False,"“What an amazing milestone."" NASA's new space ...",what an amazing milestone nasa s new space ...,"“What an amazing milestone."" NASA's new space ...","[“What, an, amazing, milestone."", NASA's, new,...",35
3,1479875700530290692,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:36,"<a href=""http://twitter.com/download/android"" ...",None,None,Twitter for Android,,,,,,,,,[[]],{},,,,,,,Cynthia Simmons,cyntsimmons,209,265,0,12120,4119,False,False,False,2013-04-11 01:00:46,None,no,https://t.co/WWhOOqSn0w,,,1,0,0,"""The observatory launched April 24, 1990, Hubb...",True,False,False,False,False,False,False,False,False,"""The observatory launched April 24, 1990, Hubb...",the observatory launched april hubb...,"""The observatory launched April 24, 1990, Hubb...","[""The, observatory, launched, April, 24,, 1990...",37
4,1479875703411728391,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:37,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,Twitter Web App,,,,,,,,,[[]],{},,,,,,,ari4,a4floridacajun,2193,3683,6,69635,72063,False,False,False,2019-01-24 00:38:11,"Niceville, FL",no,https://t.co/Kv57EeVruf,,,1,0,0,"“What an amazing milestone."" NASA's new space ...",False,True,True,False,False,F

CPU times: user 210 ms, sys: 18.4 ms, total: 228 ms
Wall time: 20.1 s


With data processing and filtering now complete, we'll update the datatype for the processed text columns that were added in this notebook

In [21]:
dtypes_dict.update(
    {
        "text_trimmed": pd.StringDtype(),
        "text_stripped": pd.StringDtype(),
        "text_processed": pd.StringDtype(),
        "words": pd.StringDtype(),
        "num_words": pd.Int32Dtype(),
    }
)

In [22]:
%%time
df_words_long = df_words_long.astype(dtypes_dict)
with pd.option_context("display.max_rows", None):
    display(df_words_long.dtypes.rename('dtype').to_frame())

,dtype
id,string
geo,string
coordinates,string
place,string
contributors,string
is_quote_status,string
quote_count,Int32
reply_count,Int32
retweet_count,Int32
favorite_count,Int32


CPU times: user 274 ms, sys: 57 ms, total: 331 ms
Wall time: 691 ms


## Export Processed Data

The data is now processed and ready for machine learning model development using big-data ML frameworks. This processed data will now be exported to a separate `.parquet` file and then uploaded to the S3 bucket. The filepath of the `.parquet` file is shown below

In [23]:
filepath = f"data/processed/{processed_file_name}.parquet.gzip"
print(filepath)

data/processed/processed_text.parquet.gzip


### Save Processed and Filtered Data to `.parquet` File

In [24]:
%%time
df_words_long.set_index('id').to_parquet(filepath, index_col='id', compression='gzip')

CPU times: user 32.7 ms, sys: 13.5 ms, total: 46.2 ms
Wall time: 25.7 s


Next, we will demonstrate loading this `.parquet` file with processed data into a `DataFrame` using two big-data frameworks - PySpark and Dask.

### Demonstrate Reloading Saved `.parquet` File with PySpark

In [25]:
%%time
df_words_long_reloaded = spark.read.parquet(filepath)
with pd.option_context("display.max_columns", None):
    display(show_pyspark_df(df_words_long_reloaded, 2))

,id,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,favorited,retweeted,created_at,source,in_reply_to_user_id,in_reply_to_screen_name,source_text,place_id,place_url,place_place_type,place_name,place_full_name,place_country_code,place_country,place_bounding_box_type,place_bounding_box_coordinates,place_attributes,coords_type,coords_lon,coords_lat,geo_type,geo_lon,geo_lat,user_name,user_screen_name,user_followers,user_friends,user_listed,user_favourites,user_statuses,user_protected,user_verified,user_contributors_enabled,user_joined,user_location,retweeted_tweet,tweet_text_urls,tweet_text_hashtags,tweet_text_usernames,num_urls_in_tweet_text,num_users_in_tweet_text,num_hashtags_in_tweet_text,text,contains_wanted_text,contains_wanted_text_case_sensitive,contains_multi_word_wanted_text,contains_crypto_terms,contains_religious_terms,contains_inappropriate_terms,contains_video_games_terms,contains_misc_unwanted_terms,contains_non_english_terms,text_stripped,text_processed,text_trimmed,words,num_words
0,1479875656062222347,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:25,"<a href=""http://twitter.com/download/iphone"" r...",None,None,Twitter for iPhone,,,,,,,,,[[]],{},,,,,,,Scherri Evans 🪐,ScherriEvans,180,209,3,21706,14009,False,False,False,2015-06-09 18:12:49,Mars,no,https://t.co/fvYJuLrkKE|https://t.co/4ovbJKdU1F,,NASAWebb,2,1,0,Today the first mirror wing on swung into pla...,False,False,True,False,False,False,False,False,False,Today the first mirror wing on swung into plac...,today the first mirror wing on swung into plac...,Today the first mirror wing on swung into pla...,"[Today, the, first, mirror, wing, on, swung, i...",41
1,1479875662878019585,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:27,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,Twitter Web App,,,,,,,,,[[]],{},,,,,,,MURTWITNESSONE,MURTWITNESSONE,3506,5002,69,521,350862,False,False,False,2009-03-19 22:25:26,"Orlando, FL",no,,,,0,0,0,Elon Musk gains 117k Twitter followers per day...,False,True,False,False,False,False,False,False,False,Elon Musk gains 117k Twitter followers per day...,elon musk gains k twitter followers per day...,Elon Musk gains 117k Twitter followers per day...,"[Elon, Musk, gains, 117k, Twitter, followers, ...",38


CPU times: user 98.8 ms, sys: 0 ns, total: 98.8 ms
Wall time: 434 ms


Get a `DataFrame` version of the Spark Schema (`df.printSchema()`) for the PySpark `DataFrame`

In [26]:
%%time
df_dtypes_pyspark = pd.DataFrame.from_records(
    [
        {"name": field.name, "dtype": field.dataType, "nullable": field.nullable}
        for field in df_words_long_reloaded.schema.fields
    ]
).set_index("name")
with pd.option_context("display.max_rows", None):
    display(df_dtypes_pyspark)

,dtype,nullable
name,,
id,StringType(),True
geo,StringType(),True
coordinates,StringType(),True
place,StringType(),True
contributors,StringType(),True
is_quote_status,StringType(),True
quote_count,IntegerType(),True
reply_count,IntegerType(),True
retweet_count,IntegerType(),True


CPU times: user 7.5 ms, sys: 3.57 ms, total: 11.1 ms
Wall time: 12 ms


### Demonstrate Reloading Saved `.parquet` File with `pandas` API on PySpark

In [27]:
%%time
df_words_long_reloaded_ps = ps.read_parquet(filepath, index_col="id").reset_index().astype(dtypes_dict)
with pd.option_context("display.max_columns", None):
    display(df_words_long_reloaded_ps.head(2))
with pd.option_context("display.max_rows", None):
    display(df_words_long_reloaded_ps.dtypes.rename('dtype').to_frame())

,id,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,favorited,retweeted,created_at,source,in_reply_to_user_id,in_reply_to_screen_name,source_text,place_id,place_url,place_place_type,place_name,place_full_name,place_country_code,place_country,place_bounding_box_type,place_bounding_box_coordinates,place_attributes,coords_type,coords_lon,coords_lat,geo_type,geo_lon,geo_lat,user_name,user_screen_name,user_followers,user_friends,user_listed,user_favourites,user_statuses,user_protected,user_verified,user_contributors_enabled,user_joined,user_location,retweeted_tweet,tweet_text_urls,tweet_text_hashtags,tweet_text_usernames,num_urls_in_tweet_text,num_users_in_tweet_text,num_hashtags_in_tweet_text,text,contains_wanted_text,contains_wanted_text_case_sensitive,contains_multi_word_wanted_text,contains_crypto_terms,contains_religious_terms,contains_inappropriate_terms,contains_video_games_terms,contains_misc_unwanted_terms,contains_non_english_terms,text_stripped,text_processed,text_trimmed,words,num_words
0,1479875656062222347,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:25,"<a href=""http://twitter.com/download/iphone"" r...",None,None,Twitter for iPhone,,,,,,,,,[[]],{},,,,,,,Scherri Evans 🪐,ScherriEvans,180,209,3,21706,14009,False,False,False,2015-06-09 18:12:49,Mars,no,https://t.co/fvYJuLrkKE|https://t.co/4ovbJKdU1F,,NASAWebb,2,1,0,Today the first mirror wing on swung into pla...,False,False,True,False,False,False,False,False,False,Today the first mirror wing on swung into plac...,today the first mirror wing on swung into plac...,Today the first mirror wing on swung into pla...,"[Today, the, first, mirror, wing, on, swung, i...",41
1,1479875662878019585,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:27,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,Twitter Web App,,,,,,,,,[[]],{},,,,,,,MURTWITNESSONE,MURTWITNESSONE,3506,5002,69,521,350862,False,False,False,2009-03-19 22:25:26,"Orlando, FL",no,,,,0,0,0,Elon Musk gains 117k Twitter followers per day...,False,True,False,False,False,False,False,False,False,Elon Musk gains 117k Twitter followers per day...,elon musk gains k twitter followers per day...,Elon Musk gains 117k Twitter followers per day...,"[Elon, Musk, gains, 117k, Twitter, followers, ...",38


,dtype
id,string
geo,string
coordinates,string
place,string
contributors,string
is_quote_status,string
quote_count,Int32
reply_count,Int32
retweet_count,Int32
favorite_count,Int32


CPU times: user 576 ms, sys: 44.3 ms, total: 620 ms
Wall time: 3.38 s


### Demonstrate Reloading Saved `.parquet` File with `dask`

In [28]:
%%time
df_words_long_reloaded_dd = dd.read_parquet(filepath).astype(dtypes_dict)
with pd.option_context("display.max_columns", None):
    display(df_words_long_reloaded_dd.head(2))
with pd.option_context("display.max_rows", None):
    display(df_words_long_reloaded_dd.dtypes.rename('dtype').to_frame())

,id,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,favorited,retweeted,created_at,source,in_reply_to_user_id,in_reply_to_screen_name,source_text,place_id,place_url,place_place_type,place_name,place_full_name,place_country_code,place_country,place_bounding_box_type,place_bounding_box_coordinates,place_attributes,coords_type,coords_lon,coords_lat,geo_type,geo_lon,geo_lat,user_name,user_screen_name,user_followers,user_friends,user_listed,user_favourites,user_statuses,user_protected,user_verified,user_contributors_enabled,user_joined,user_location,retweeted_tweet,tweet_text_urls,tweet_text_hashtags,tweet_text_usernames,num_urls_in_tweet_text,num_users_in_tweet_text,num_hashtags_in_tweet_text,text,contains_wanted_text,contains_wanted_text_case_sensitive,contains_multi_word_wanted_text,contains_crypto_terms,contains_religious_terms,contains_inappropriate_terms,contains_video_games_terms,contains_misc_unwanted_terms,contains_non_english_terms,text_stripped,text_processed,text_trimmed,words,num_words
0,1479875656062222347,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:25,"<a href=""http://twitter.com/download/iphone"" r...",None,None,Twitter for iPhone,,,,,,,,,[[]],{},,,,,,,Scherri Evans 🪐,ScherriEvans,180,209,3,21706,14009,False,False,False,2015-06-09 18:12:49,Mars,no,https://t.co/fvYJuLrkKE|https://t.co/4ovbJKdU1F,,NASAWebb,2,1,0,Today the first mirror wing on swung into pla...,False,False,True,False,False,False,False,False,False,Today the first mirror wing on swung into plac...,today the first mirror wing on swung into plac...,Today the first mirror wing on swung into pla...,"[Today, the, first, mirror, wing, on, swung, i...",41
1,1479875662878019585,None,None,None,None,False,0,0,0,0,False,False,2022-01-08 18:00:27,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,Twitter Web App,,,,,,,,,[[]],{},,,,,,,MURTWITNESSONE,MURTWITNESSONE,3506,5002,69,521,350862,False,False,False,2009-03-19 22:25:26,"Orlando, FL",no,,,,0,0,0,Elon Musk gains 117k Twitter followers per day...,False,True,False,False,False,False,False,False,False,Elon Musk gains 117k Twitter followers per day...,elon musk gains k twitter followers per day...,Elon Musk gains 117k Twitter followers per day...,"[Elon, Musk, gains, 117k, Twitter, followers, ...",38


,dtype
id,string
geo,string
coordinates,string
place,string
contributors,string
is_quote_status,string
quote_count,Int32
reply_count,Int32
retweet_count,Int32
favorite_count,Int32


CPU times: user 550 ms, sys: 117 ms, total: 667 ms
Wall time: 661 ms


## Zip and Upload Processed Data `.parquet` Files to S3 Bucket

We'll now
- create a `.zip` file from the `.parquet` file (PySpark creates a folder of `.parquet` files, not a single file like `pandas` did in the previous notebook)
- upload the `.zip` file to S3 in the `/processed` prefix
- delete the local `.parquet` folder with processed data
- delete the local `.zip` file prepared from the `.parquet` folder

In [29]:
print(filepath)
print(path_to_folder)
print(processed_data_dir)
print(proc_text_zip_fname)
print(f"{path_to_folder[1:-1]}/processed/{proc_text_zip_fname}")
print(os.path.join(processed_data_dir, proc_text_zip_fname))

data/processed/processed_text.parquet.gzip
/datasets/twitter/kinesis-demo/
data/processed
processed_text.zip
datasets/twitter/kinesis-demo/processed/processed_text.zip
data/processed/processed_text.zip


In [31]:
%%time
if upload_to_s3:
    # create zip of all .parquet.gzip processed data files
    create_zip_file_from_folder(processed_data_dir, proc_text_zip_fname, filepath)

    # upload zip file to S3 bucket
    upload_file_to_s3(
        session.region_name,
        processed_data_dir,
        proc_text_zip_fname,
        s3_bucket_name,
        f"{path_to_folder[1:-1]}/processed/{proc_text_zip_fname}",
    )
    print("\nUploaded zipped file to S3 bucket")

if cleanup_local_files:
    # delete folder with locally exported parquet files containing processed data
    # exported by PySpark
    shutil.rmtree(filepath)
    print("Deleted local .parquet.gzip files with processed data.")
    # delete local zip file
    if upload_to_s3:
        os.remove(os.path.join(processed_data_dir, proc_text_zip_fname))
        print("Deleted local .zip file created from all filtered data files.")


Uploaded zipped file to S3 bucket
Deleted local .parquet.gzip files with processed data.
Deleted local .zip file created from all filtered data files.
CPU times: user 381 ms, sys: 304 ms, total: 685 ms
Wall time: 894 ms


## Cleanup

We'll now
- delete the local `.parquet` folder with prepared data that was processed in this notebook
- delete the local `.zip` file (containing the individual `.parquet` files of prepared data) that was downloaded from S3

In [32]:
print(os.path.join(processed_data_dir, proc_zip_fname))
proc_files[:5]

data/processed/processed_data.zip


['data/processed/filtered__2022010401.parquet.gzip',
 'data/processed/filtered__2021123017.parquet.gzip',
 'data/processed/filtered__2021123118.parquet.gzip',
 'data/processed/filtered__2021123119.parquet.gzip',
 'data/processed/filtered__2021123120.parquet.gzip']

In [33]:
%%time
if cleanup_local_files:
    # delete locally exported parquet files
    _ = list(map(os.remove, proc_files))
    print("Deleted local .parquet.gzip files with filtered data.")

    # delete local zip file
    os.remove(os.path.join(processed_data_dir, proc_zip_fname))
    print("Deleted local .zip file created from all filtered data files.")

Deleted local .parquet.gzip files with filtered data.
Deleted local .zip file created from all filtered data files.
CPU times: user 224 µs, sys: 13.8 ms, total: 14.1 ms
Wall time: 14.1 ms


---

<span style="float:left;">
    <a href="./4_filter_data.ipynb"><< 4 - Filter Data</a>
</span>

<span style="float:right;">
    <a href="./6_split_data.ipynb">6 - Create Data Splits >></a>
</span>